In [1]:
import csv                       #use library csv to open and read csv data
import numpy as np 
import math
import datetime
import random
np.set_printoptions(threshold=np.inf)

path='/home/mz/Desktop/breast-cancer-wisconsin.data'

with open(path,'r') as cancerdata:
    reader = csv.reader(cancerdata) 
    rows= [row for row in reader]
    
cdata=np.array(rows)               #change rows' datatype into array

todelete=np.where(cdata=='?')               #save row number of 16 arrays which need to be deleted
#print(todelete[0])
ddataset=np.delete(cdata, todelete[0], axis = 0) #delete those rows which contain ‘？’

#print("The type and shape of cdata is",type(cdata), cdata.shape)
#print("The type and shape of ddataset is",type(ddataset), ddataset.shape)
test=np.where(ddataset=='?')                #Check if there are still '?' in ndarray ddataset
#print("The new dataset is contains ? in ", test)
#print(ddataset)
#print('\n')
cdataset = ddataset.astype(np.float) #change datatype from string to float
#print(cdataset)

dataset = np.delete(cdataset,0,axis = 1) #delete first row
for i in range(dataset.shape[0]):
    if dataset[i,dataset.shape[1]-1]==2: dataset[i,dataset.shape[1]-1]=0
    else: dataset[i,dataset.shape[1]-1]=1
#print(dataset,dataset.shape)
#data0=dataset
datash = dataset
np.random.shuffle(datash)
data0=datash

k=5
n0 = data0.shape[0]
X_test = data0[0:int(n0/k), 0:-1 ]
y_test = data0[0:int(n0/k), -1 ]

data2 = np.delete(data0, range(0,int(n0/k)),axis = 0)#delete testset from data
X_all = data2[:,0:-1]
y_all = data2[:,-1]


n1 = data2.shape[0]#number of rows in data2 for training and verti
X_valid = [[]for i in range(k)]
X_train = [[]for i in range(k)]
y_valid = [[]for i in range(k)]
y_train = [[]for i in range(k)]
trainset = [[]for i in range(k)]
for i in range(0,k):
    X_valid[i] = np.array(data2[int((i*n1)/k):int((i+1)*n1/k), 0:-1])
    y_valid[i] = np.array(data2[int((i*n1)/k):int((i+1)*n1/k), -1])
    a=np.arange(int((i*n1)/k),int((i+1)*n1/k))
    trainset[i] = np.delete(data2, a , axis=0)
    X_train[i] = trainset[i][:,0:-1]
    y_train[i] = trainset[i][:,-1]
    print('valid i=', i, X_train[i].shape)
    print('valid i=', i, y_train[i].shape)

valid i= 0 (438, 9)
valid i= 0 (438,)
valid i= 1 (438, 9)
valid i= 1 (438,)
valid i= 2 (437, 9)
valid i= 2 (437,)
valid i= 3 (438, 9)
valid i= 3 (438,)
valid i= 4 (437, 9)
valid i= 4 (437,)


In [11]:
from sklearn.linear_model import LogisticRegression
X = X_all
y = y_all
clf = LogisticRegression().fit(X, y)
r = clf.predict(X_test)
accuracy(r,y_test)

/home/mz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9558823529411765

In [4]:
def sigmoid(gamma):#the sigmoid function we used in gradient descenting and prediction. To avoid overflowing, input of exp is always a negative number
    if gamma < 0:
        return 1 - 1/(1 + math.exp(gamma))
    else:
        return 1/(1 + math.exp(-gamma))
    

'''
def add(data,w):
            m = data.shape[1]
            summe = np.zeros(m-1)
            for i in range(0,data.shape[0]):
                a = data[i,0:(data.shape[1]-1)]*((data[i,(data.shape[1]-1)])-sigma(w, data[i,0:(data.shape[1]-1)]))
                summe = summe + a         
            return summe    
    
def gd(w,lr,data):
            neww = w + lr*add(data,w)
            return neww
'''
#lr learning rate; n iteration times; data data used for training; y_train:label of training data
def findw(lr,n,data,y_train):
    traindata = data
    m = traindata.shape[1]#column number of input data(features)
    #w = [0,0,0,0,0,0,0,0,0,0,0]#除去最后一列的quality
    #w = [1,1,1,1,1,1,1,1,1,1,1]
    w = np.ones(m)#define w
    #right = [0,0,0,0,0,0,0,0,0,0,0]
    #vector
    right = np.zeros(m)
    for i in range(1,n):#n times iteration
        
        for j in range(0,traindata.shape[0]):
            right = right + traindata[j,]*(y_train[j]-sigmoid(np.dot(w,traindata[j,])))
            
        for l in range(0,right.shape[0]):
            right[l]=right[l]*lr
        #print(right)
        w=w+right
        #print(w)
    return w


#data.TEST DATA; w:w from findw function
def predict(data,w):#use w which calculated by findw function to predict the classification of input data
        n = data.shape[0]
        result = np.zeros(n)
        for i in range(0,n):
            prob = sigmoid(np.dot(w,data[i,]))
            #prob = (1/math.sqrt(2*pi*sigma(w,data[i,0:n-1]))) * math.exp(-(1-np.transpose(w)*data[i,0:n-1]/2(sigma(w,data[i,0:n-1])*sigma(w,data[i,0:n-1]))))
            if prob>=0.5: result[i]=1
        return result
#pred: result from predict function; y: real value(label of training data)            
def accuracy(pred, y):#pred is result from fct predict
    real = y
    
    if pred.shape != real.shape:
        print('Error!!!')
        
    acc=0
    for i in range(pred.shape[0]):
        if pred[i]==real[i]:
            acc=acc+1
    Accuracy=acc/pred.shape[0]   
    #print('Accuracy is ',Accuracy)
    return Accuracy

#result: result from predict function; y: real value
def conmat(result,y):#return confusion matrix for prediction
    rv = y
    print(result.shape)
    print(rv.shape)
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(result.shape[0]):
        if ((result[i]==1)&(rv[i]==1)): TP = TP+1
        if ((result[i]==0)&(rv[i]==0)): TN = TN+1
        if ((result[i]==0)&(rv[i]==1)): FN = FN+1
        if ((result[i]==1)&(rv[i]==0)): FP = FP+1
    metrix = np.array([[TP,FP], [FN,TN]])
    return metrix


def fit():#tuning training and testing; output all informations we need
    start = datetime.datetime.now()
    a_n = 7#number of groups of alpha
    iter_n= 5#number of n
    numwewant = 40#number of randomly chosed pairs
    #give alpha values in 1dB step
    alpha = np.zeros(9*a_n)
    for i in range(9):
        alpha[i]= 1e-7*(i+1)
        alpha[i+9]= 1e-6*(i+1)
        alpha[i+18]= 1e-5*(i+1)
        alpha[i+27]= 1e-4*(i+1)
        alpha[i+36]= 1e-3*(i+1)
        alpha[i+45]= 1e-2*(i+1)
        alpha[i+54]= 1e-1*(i+1)
        #alpha[i+63]= (i+1)
        #alpha[i+72]= (i+1)*10
        #alpha[i+81]= (i+1)*100
    
    #n_iteration = [500, 1000, 1500]
    n_iteration = [500, 1000, 1500, 2500, 3000]
    
    pairs = [[]for i in range(a_n*iter_n*9)]
    n = 0
    
    #offer all possibilities for pairs of alpha and iteration
    for i in range(9*a_n):
        j = 0
        while j<5:
            pairs[n] = [alpha[i], n_iteration[j]]
            n = n+1
            j = j+1
             
    print(pairs)  
    
    print('\n')
    #print(pairs)
    
    #here we chose numwewant from 9*a_n*5
    para_pos = np.array(random.sample(range(0,a_n*iter_n*9), numwewant))#generate used pairs's position
    #result = np.zeros([80,256])
    result = [[]for i in range(numwewant)]#save the predict results
    print(para_pos)
    A = np.zeros(numwewant)#the avg accuracy vector for 80 pairs
    Accu = np.zeros([numwewant,5])#Accuracy for each validation
    #ln=0
    
    for i in range(numwewant):
        print('\n')
        pos = para_pos[i]
        print('pos',pos)
        para = pairs[pos]
        print('para',para)
        for j in range(5):
            #a = para[0]
            #b = para[1]
            #print(a)
            #print(b)
            lr = para[0]
            n = para[1]
            w_model = findw(lr,n,X_train[j],y_train[j])#find w according to X_train and y_train
            print('fold',j,'of model',i,'finished',w_model)
            result[i] = predict(X_valid[j],w_model)#using X_valid as input to produce prediction output
            #result = predict(X_valid[j],w_model)
            #result[ln] = predict(X_valid[j],w_model)
            #ln = ln+1
            Accu[i,j]= accuracy(result[i], y_valid[j])#compare result(output prediction) and real values y_valid
            print('Accuracy is',Accu[i,j])
            if j ==4: A[i] = np.sum(Accu[i,:])/5#calculate avg accuracy
            if j ==4: print('\033[1;35m Average accuracy is \033[0m',A[i])          
            print('\n')
            
            
  


        
    a = np.max(A)#find the highst accuracy
    pos_max = np.where(A==a)#find the location of highest acc
    location = pos_max[0][0]#location is a number which used to locate which random number in random vector
    print('used position',location)
    bestalpha = pairs[para_pos[location]][0]#use location to find best alpha and n
    bestn = pairs[para_pos[location]][1]
    print('avg A is',A)
    print('max',np.max(A))
    print('max_pos',pos_max)
    print('location',location)
    print(pairs[para_pos[location]])
    print('best para are',bestalpha,bestn)
    
    w_final = findw(bestalpha, bestn, X_all, y_all)#training wo find w with found hyperparameters
    final_result = predict(X_test,w_final)#using testset to verify and produce prediction results
    finalaccu = accuracy(final_result,y_test)#calculate the accuracy according to label y_test and prediction results
    finalco = conmat(final_result,y_test)#produce confusion matrix
    print('The final accuracy is', finalaccu)
    print('The confusion matrix of prediction is\n', finalco)
    end = datetime.datetime.now()
    print ('used time is',end-start)
    return w_final





In [46]:
fit()#5th

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 3 of model 2 finished [-144.48813975  285.18433427  148.29020829   10.19606104 -325.76488479
  211.08511559 -122.17433239  108.68447538  -92.66887599]
Accuracy is 0.8807339449541285


fold 4 of model 2 finished [ -84.35525611  266.9310591   110.60291747   14.07106901 -312.13040636
  124.4361671  -198.00434488  138.26458089 -105.76271777]
Accuracy is 0.8545454545454545
 Average accuracy is  0.8264553794829024




pos 282
para [0.30000000000000004, 1500]
fold 0 of model 3 finished [-137.9647559   331.91866774  119.48647094   82.51402131 -347.30467318
  202.68599764 -202.44409158  167.75213115 -125.22555059]
Accuracy is 0.8990825688073395


fold 1 of model 3 finished [ -39.02052131  348.6534758   122.7060421   164.79791496 -289.29601639
  170.55131763  -99.95874345  181.65856711 -104.9196531 ]
Accuracy is 0.5045871559633027


fold 2 of model 3 finished [  -4.91322912  457.14526345  260.86250891  187.38656743 -139.04855473
  450.43826189  -45.2653792   300.02403697 -177.04164334]
Accu

fold 1 of model 11 finished [-48.4907324   72.9022127   19.25965777  18.84880987 -90.11583269
  32.12472355 -51.59921662  30.22790551 -36.85254592]
Accuracy is 0.8256880733944955


fold 2 of model 11 finished [-53.53087695  53.32296281  10.34558681  -2.29637314 -76.42224534
  40.28227666 -60.58929446  22.8542261  -58.52229733]
Accuracy is 0.5909090909090909


fold 3 of model 11 finished [-36.66037416  61.826259    26.83805166 -10.51964319 -88.32634129
  44.0896096  -42.32028063  19.88382019 -29.82681266]
Accuracy is 0.7064220183486238


fold 4 of model 11 finished [-30.93787554  80.75332353  43.39381526  10.5855265  -86.43541977
  51.1074648  -49.19328247  46.62290357 -28.07024698]
Accuracy is 0.8727272727272727
 Average accuracy is  0.7697914929107589




pos 240
para [0.04, 500]
fold 0 of model 12 finished [-13.43722698  53.10207264  25.06232232  14.87911447 -47.56984789
  38.12789925 -23.2422754   30.18638015 -15.27719405]
Accuracy is 0.7339449541284404


fold 1 of model 12 finished

fold 4 of model 19 finished [-2.27481281  6.84565861  3.78854712  1.18427562 -6.34683076  4.61692
 -3.25989494  4.09977159 -2.19439628]
Accuracy is 0.7727272727272727
 Average accuracy is  0.7548790658882402




pos 94
para [1e-05, 3000]
fold 0 of model 20 finished [-0.32718122  0.56977951  0.32353897  0.15434194 -0.64064516  0.45167879
 -0.44257653  0.32436342 -0.238424  ]
Accuracy is 0.8899082568807339


fold 1 of model 20 finished [-0.35411047  0.67488775  0.35827343  0.22463822 -0.70346634  0.42154921
 -0.48116235  0.3594928  -0.29885532]
Accuracy is 0.8532110091743119


fold 2 of model 20 finished [-0.35525683  0.60565244  0.28710729  0.06625467 -0.59796294  0.56836244
 -0.48871244  0.36309581 -0.41270311]
Accuracy is 0.8181818181818182


fold 3 of model 20 finished [-0.3399587   0.5969305   0.44406001 -0.02127337 -0.73655705  0.55229915
 -0.42946732  0.30390697 -0.26769518]
Accuracy is 0.8807339449541285


fold 4 of model 20 finished [-0.30307682  0.62535286  0.39259705  0.079370

fold 2 of model 28 finished [-0.36662175  0.55237673  0.29376455  0.06158302 -0.56877556  0.54029484
 -0.46724382  0.34708053 -0.29940184]
Accuracy is 0.8090909090909091


fold 3 of model 28 finished [-0.34431503  0.54810962  0.41829631 -0.02303943 -0.66651762  0.52888718
 -0.41632598  0.29309014 -0.22147401]
Accuracy is 0.8715596330275229


fold 4 of model 28 finished [-0.30623223  0.56906471  0.37294869  0.06751382 -0.65915228  0.42968592
 -0.44959608  0.37620788 -0.19915393]
Accuracy is 0.8636363636363636
 Average accuracy is  0.8538115095913261




pos 169
para [0.0007, 3000]
fold 0 of model 29 finished [-0.50083331  0.9467019   0.3784992   0.183287   -1.16247665  0.63097296
 -0.69446678  0.50017499 -0.43330574]
Accuracy is 0.8532110091743119


fold 1 of model 29 finished [-0.38572012  1.23553769  0.46193799  0.39189973 -1.12251041  0.64940099
 -0.60021803  0.60512539 -0.42840931]
Accuracy is 0.8348623853211009


fold 2 of model 29 finished [-0.48827454  0.94393264  0.28507644  0.0

fold 0 of model 37 finished [ -7.1505542   12.73967962   5.42221339   2.71295387 -13.40276634
   8.83804545  -7.88295961   6.97509228  -5.07603568]
Accuracy is 0.8899082568807339


fold 1 of model 37 finished [ -5.60195354  12.46652579   4.33328094   4.42138076 -13.04883222
   6.33310584  -6.79635128   5.84402717  -5.54784998]
Accuracy is 0.8623853211009175


fold 2 of model 37 finished [-5.39605356 11.78198953  4.71126099  1.90852017 -9.8880592   9.64014455
 -7.26398146  6.37169627 -8.52098028]
Accuracy is 0.8636363636363636


fold 3 of model 37 finished [ -4.76493727  11.77335502   6.62232318   0.10131128 -12.73457475
   9.34285459  -5.4954052    5.0323517   -4.17900354]
Accuracy is 0.8715596330275229


fold 4 of model 37 finished [ -3.23756292  10.83457337   5.47101914   1.3580791  -11.30889467
   6.28234352  -6.55231209   6.03653912  -3.73265072]
Accuracy is 0.8818181818181818
 Average accuracy is  0.8738615512927439




pos 255
para [0.07, 500]
fold 0 of model 38 finished [ -61.69

array([ -6.45602143,  15.65609366,   7.55684648,   3.03316236,
       -14.43374536,  10.84624239,  -7.86365554,   8.40744024,
        -6.02971314])

In [42]:
fit()#4th

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 4 of model 2 finished [0.66108328 0.85110139 0.83593186 0.84351669 0.75769078 0.84311739
 0.76048444 0.85150062 0.87625098]
Accuracy is 0.34545454545454546
 Average accuracy is  0.34173477898248544




pos 92
para [1e-05, 1500]
fold 0 of model 3 finished [-0.35819377  0.52248239  0.2733627   0.12550595 -0.49485788  0.42330633
 -0.43225859  0.29348819 -0.08730703]
Accuracy is 0.9174311926605505


fold 1 of model 3 finished [-0.38535672  0.54810128  0.32927585  0.10721336 -0.49820774  0.39478432
 -0.39692307  0.2618924  -0.14412132]
Accuracy is 0.8440366972477065


fold 2 of model 3 finished [-0.35015944  0.57069496  0.28786146  0.09170787 -0.57041561  0.45099486
 -0.4277679   0.24687166 -0.11467197]
Accuracy is 0.8454545454545455


fold 3 of model 3 finished [-0.38851788  0.54575225  0.29422818  0.07216177 -0.54647883  0.46336358
 -0.41831922  0.3164057  -0.09324008]
Accuracy is 0.8440366972477065


fold 4 of model 3 finished [-0.35864288  0.56463659  0.2511245   0.08584737 -0.5294

fold 2 of model 11 finished [-330.08247113  503.34647219  117.58177777   39.48681096 -452.40798789
  331.84014197 -308.71981637  181.87304699 -186.99839646]
Accuracy is 0.8272727272727273


fold 3 of model 11 finished [  -3.11263828  396.51046826  103.27523558   88.76435346 -195.42745057
  237.763476    -91.64483133  190.01076481  -80.65858584]
Accuracy is 0.3486238532110092


fold 4 of model 11 finished [-110.46890898  403.80172887  -16.92495642   39.04454429 -263.65354386
  124.86075895 -177.83491438  135.88791464 -157.45338957]
Accuracy is 0.8272727272727273
 Average accuracy is  0.7345788156797332




pos 302
para [0.7000000000000001, 1500]
fold 0 of model 12 finished [-204.20599911  721.85281993 -115.72853582   14.75754828 -504.17914581
  208.8491847  -382.9797178   248.89952438 -254.11781211]
Accuracy is 0.8440366972477065


fold 1 of model 12 finished [-124.90300741  823.54746915   53.37717451  168.36985265 -496.0836453
  258.74235282 -350.12690289  359.34018289 -282.31110608]
A

fold 4 of model 19 finished [-0.3501347   0.89868195  0.12777401  0.13480418 -0.66619069  0.46947056
 -0.50968482  0.40409845 -0.38310859]
Accuracy is 0.8545454545454545
 Average accuracy is  0.8665888240200166




pos 184
para [0.001, 3000]
fold 0 of model 20 finished [-0.47691253  2.02180835  0.392579    0.42583947 -1.38947308  1.08831215
 -0.85989368  0.87686665 -0.43800616]
Accuracy is 0.7706422018348624


fold 1 of model 20 finished [-0.78677139  1.77557379  0.3271188   0.12750911 -1.47052726  0.75907575
 -0.92473134  0.62861838 -0.72578184]
Accuracy is 0.8256880733944955


fold 2 of model 20 finished [-0.73246188  1.82277922  0.28247029  0.10377716 -1.68435134  0.98972876
 -1.04741575  0.53647053 -0.54772465]
Accuracy is 0.8272727272727273


fold 3 of model 20 finished [-0.71879196  1.62642504  0.31299052  0.09037759 -1.51331223  0.92250696
 -0.97588194  0.7035015  -0.69151257]
Accuracy is 0.8440366972477065


fold 4 of model 20 finished [-0.69069003  1.8512249   0.08672246  0.06

fold 2 of model 28 finished [-0.66275332  1.68526866  0.25253501  0.10184515 -1.53741789  0.89898824
 -0.96294545  0.49467284 -0.50897097]
Accuracy is 0.8363636363636363


fold 3 of model 28 finished [-0.65424273  1.49777658  0.28428552  0.08396168 -1.3693212   0.83962534
 -0.89925165  0.64792761 -0.62873045]
Accuracy is 0.8532110091743119


fold 4 of model 28 finished [-0.62791031  1.69501896  0.07046807  0.06939617 -1.24257368  0.74581392
 -0.87489387  0.59610038 -0.78135107]
Accuracy is 0.8181818181818182
 Average accuracy is  0.8520100083402836




pos 55
para [3e-06, 500]
fold 0 of model 29 finished [-0.20234386  0.44971718  0.40793876  0.43037879  0.13699812  0.42578019
  0.11451382  0.4542102   0.560406  ]
Accuracy is 0.24770642201834864


fold 1 of model 29 finished [-0.31285567  0.43211179  0.38446634  0.3978951   0.09396912  0.40212107
  0.08491989  0.45453561  0.53517523]
Accuracy is 0.3669724770642202


fold 2 of model 29 finished [-0.29213048  0.40982108  0.35470958  0.399

fold 0 of model 37 finished [-0.38406442  0.42890775  0.27061121  0.12168584 -0.43478946  0.40387528
 -0.38088053  0.26821462  0.02572355]
Accuracy is 0.908256880733945


fold 1 of model 37 finished [-0.42072959  0.46089817  0.30880532  0.10519705 -0.43080415  0.3778151
 -0.35067173  0.25127165 -0.01596147]
Accuracy is 0.8532110091743119


fold 2 of model 37 finished [-0.38228755  0.46243621  0.27544751  0.09178623 -0.48221828  0.43167097
 -0.37945269  0.23507437 -0.00634625]
Accuracy is 0.8363636363636363


fold 3 of model 37 finished [-0.42264557  0.45254133  0.28597495  0.08062207 -0.46781551  0.43122205
 -0.37418845  0.29316459  0.03319425]
Accuracy is 0.8165137614678899


fold 4 of model 37 finished [-0.38692244  0.46836886  0.25599634  0.08885662 -0.46601757  0.39370844
 -0.34395424  0.27788182 -0.03451873]
Accuracy is 0.8636363636363636
 Average accuracy is  0.8555963302752293




pos 54
para [2e-06, 3000]
fold 0 of model 38 finished [-0.44363746  0.34091789  0.2481923   0.14321

array([-0.36052351,  1.04707266,  0.13311252,  0.15066681, -0.77416629,
        0.50765737, -0.57164803,  0.4033205 , -0.35161711])

In [38]:
fit()#3rd

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 0 of model 3 finished [-32.8363612   88.90778183   5.72914253  29.44455712 -79.84509759
  56.99051459 -54.77383489  35.32473274 -14.91417553]
Accuracy is 0.8165137614678899


fold 1 of model 3 finished [  0.55864445 105.91347372  52.62816179  43.02435049 -47.86380738
  83.3211109  -11.41259168  35.24715705  -3.58019114]
Accuracy is 0.3577981651376147


fold 2 of model 3 finished [-25.89135656  66.61447263  13.47822042   7.43625901 -60.04364698
  28.77007079 -41.91401015  18.14779589 -20.39676573]
Accuracy is 0.8363636363636363


fold 3 of model 3 finished [-46.32180386  97.16354813  26.47514253  14.48143124 -84.04596075
  54.73364112 -50.71707771  22.75449449 -45.2417204 ]
Accuracy is 0.9174311926605505


fold 4 of model 3 finished [ -1.89662975 139.48388514  38.93043077  34.94114949 -43.0626096
  89.823374   -20.53538983  55.58518546 -10.30005131]
Accuracy is 0.36363636363636365
 Average accuracy is  0.6583486238532111




pos 79
para [7e-06, 3000]
fold 0 of model 4 finished [-0.

fold 4 of model 11 finished [-0.34838391  1.08329654  0.12927428  0.14355348 -0.68676935  0.55593048
 -0.60963299  0.25298589 -0.25976747]
Accuracy is 0.8454545454545455
 Average accuracy is  0.8574145120934112




pos 145
para [0.00030000000000000003, 500]
fold 0 of model 12 finished [-0.34909692  1.02915827  0.02462793  0.34578423 -0.84276592  0.62741468
 -0.67934032  0.38781368 -0.20072275]
Accuracy is 0.8440366972477065


fold 1 of model 12 finished [-0.38668775  0.87242603  0.32896818  0.24347885 -0.77142195  0.62393558
 -0.55654449  0.13615927 -0.21215236]
Accuracy is 0.8623853211009175


fold 2 of model 12 finished [-0.31506562  0.82184749  0.22591022  0.15740758 -0.69162798  0.48843467
 -0.53976595  0.28014434 -0.26756748]
Accuracy is 0.8636363636363636


fold 3 of model 12 finished [-0.33504451  0.97597148  0.25640888  0.20758187 -0.71679205  0.49892792
 -0.5389832   0.23930762 -0.37718813]
Accuracy is 0.8899082568807339


fold 4 of model 12 finished [-0.35209701  1.17913986  

fold 2 of model 20 finished [-0.52176254  0.31458362  0.23959471  0.17148325 -0.29778098  0.32982362
 -0.25108891  0.24512371  0.25376225]
Accuracy is 0.8


fold 3 of model 20 finished [-0.49157114  0.3228292   0.25495261  0.17552915 -0.33140596  0.33714996
 -0.2378781   0.2259406   0.26990628]
Accuracy is 0.8073394495412844


fold 4 of model 20 finished [-0.5057393   0.3387911   0.21767059  0.16865711 -0.3191175   0.37012173
 -0.26219454  0.24609738  0.26898966]
Accuracy is 0.7909090909090909
 Average accuracy is  0.7787322768974145




pos 304
para [0.7000000000000001, 3000]
fold 0 of model 21 finished [-185.49833036  726.26657774 -225.06570329  202.86418471 -577.67856784
  383.9407904  -519.97808558  215.00860357  -70.91129067]
Accuracy is 0.7981651376146789


fold 1 of model 21 finished [-554.86106094 1027.05445909  598.13847971  351.3654374  -738.00614693
 1006.07869228 -408.98783778  315.30273728 -305.81191247]
Accuracy is 0.6880733944954128


fold 2 of model 21 finished [ -88.85

fold 0 of model 29 finished [-233.57508202  507.67601518   75.24001326  176.83889124 -468.63546918
  361.64562495 -303.40915818  231.35780615 -118.39085886]
Accuracy is 0.8348623853211009


fold 1 of model 29 finished [ -87.13653698  492.32696071  261.82720983  206.87561112 -269.01840283
  425.04553608  -77.17783846  181.23095735  -45.2025534 ]
Accuracy is 0.3577981651376147


fold 2 of model 29 finished [-139.16143913  429.34995388  108.4259305    79.85489306 -347.44621578
  222.85362379 -226.26422123  153.55090668 -103.69211631]
Accuracy is 0.8727272727272727


fold 3 of model 29 finished [-134.67893723  443.90824604   97.14770558   61.38981877 -307.46865973
  192.4188793  -218.58895596   89.91501372 -176.14579225]
Accuracy is 0.8990825688073395


fold 4 of model 29 finished [-236.44820637  596.47362379  109.59536738   67.83629347 -373.8120588
  374.62744727 -278.54099389  206.89122148 -157.05846955]
Accuracy is 0.8818181818181818
 Average accuracy is  0.7692577147623019




pos 163


fold 3 of model 37 finished [0.58384158 0.8158744  0.79840957 0.8068926  0.69462122 0.81138339
 0.70460014 0.81637346 0.85130339]
Accuracy is 0.3119266055045872


fold 4 of model 37 finished [0.57186555 0.81487639 0.7929206  0.8048965  0.69512015 0.8178704
 0.70110745 0.81886843 0.84781038]
Accuracy is 0.36363636363636365
 Average accuracy is  0.34356964136780654




pos 265
para [0.09, 500]
fold 0 of model 38 finished [ -36.373192    121.95001187    6.9445252    40.18003137 -103.91471504
   77.50919663  -70.34570243   44.28656094  -17.48427281]
Accuracy is 0.8165137614678899


fold 1 of model 38 finished [ -47.9004679   111.97512598   30.80050978   26.21767458 -108.50785828
   68.38685231  -60.75362658   13.0760238   -23.23913696]
Accuracy is 0.8440366972477065


fold 2 of model 38 finished [ -43.9421611   120.75542261   33.84253484   21.42840838 -100.46725106
   65.404372    -64.40298004   42.79999425  -32.0744167 ]
Accuracy is 0.8454545454545455


fold 3 of model 38 finished [ -53.6

array([-0.367558  ,  1.46926394,  0.34006577,  0.32199452, -1.0234815 ,
        0.84304624, -0.68372141,  0.42888273, -0.35118253])

In [39]:
fit()#2nd

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 4 of model 2 finished [-0.33869198  0.81729238  0.22186498  0.13676878 -0.61605705  0.50558817
 -0.52222121  0.23245086 -0.1789635 ]
Accuracy is 0.8454545454545455
 Average accuracy is  0.8519432860717264




pos 270
para [0.1, 500]
fold 0 of model 3 finished [ -49.92908416  144.20784159   10.31166197   48.00919671 -128.59454482
   92.42368523  -89.63219101   59.02572554  -22.43302957]
Accuracy is 0.8256880733944955


fold 1 of model 3 finished [ -60.23394754  130.94722428   45.52009765   34.86491609 -128.86904084
   87.2234259   -68.94497075   18.09373809  -29.57493383]
Accuracy is 0.8623853211009175


fold 2 of model 3 finished [  6.76229707 137.52598633  48.9011258   37.97884351 -68.27000256
  81.13550925 -32.22253693  55.21183289 -15.28024014]
Accuracy is 0.4


fold 3 of model 3 finished [ -45.53541315  129.86262798   22.52080001   14.03087182 -103.5453231
   49.80911671  -69.12895057   19.22837417  -58.54492591]
Accuracy is 0.8623853211009175


fold 4 of model 3 finished [ -4

fold 2 of model 11 finished [-0.56247087  0.29546404  0.22840091  0.27384957 -0.12405067  0.26008918
 -0.12089203  0.31757179  0.42499315]
Accuracy is 0.6454545454545455


fold 3 of model 11 finished [-0.52840362  0.30928596  0.24478062  0.27436616 -0.14275104  0.29229309
 -0.10312366  0.3089056   0.44380586]
Accuracy is 0.6330275229357798


fold 4 of model 11 finished [-0.5556416   0.31162669  0.22935466  0.27212574 -0.13194871  0.32309163
 -0.10943745  0.32322061  0.43436312]
Accuracy is 0.6545454545454545
 Average accuracy is  0.6288073394495413




pos 51
para [2e-06, 1000]
fold 0 of model 12 finished [-0.52580963  0.30364868  0.20989933  0.31037065 -0.13578989  0.28722803
 -0.11462892  0.31895723  0.43276568]
Accuracy is 0.5963302752293578


fold 1 of model 12 finished [-0.57882385  0.29004419  0.25057462  0.28563011 -0.15335197  0.31897414
 -0.1213912   0.29422034  0.43186011]
Accuracy is 0.6146788990825688


fold 2 of model 12 finished [-0.56259173  0.29529193  0.22822366  0.273

fold 0 of model 20 finished [-0.38454656  0.42654517  0.212778    0.2352061  -0.50207414  0.42576979
 -0.37905257  0.25541222  0.08186508]
Accuracy is 0.8440366972477065


fold 1 of model 20 finished [-0.4280305   0.39858236  0.32224202  0.20553054 -0.48720039  0.46803414
 -0.31807932  0.11451243  0.09066772]
Accuracy is 0.7981651376146789


fold 2 of model 20 finished [-0.41928722  0.40372602  0.29397553  0.11828638 -0.43775476  0.40515949
 -0.33344937  0.22404855  0.06547656]
Accuracy is 0.8454545454545455


fold 3 of model 20 finished [-0.38826264  0.41926448  0.31411225  0.13909084 -0.46745965  0.39984079
 -0.32052056  0.20079448  0.06849515]
Accuracy is 0.8715596330275229


fold 4 of model 20 finished [-0.39139764  0.46045119  0.25185737  0.12166668 -0.45461055  0.43478309
 -0.3618338   0.22221881  0.08397335]
Accuracy is 0.8545454545454545
 Average accuracy is  0.8427522935779816




pos 110
para [5e-05, 500]
fold 0 of model 21 finished [-0.33798909  0.68331913  0.17808602  0.294

fold 4 of model 28 finished [-0.35282055  1.19568812  0.08671256  0.13910919 -0.71973532  0.5770804
 -0.63818357  0.26166476 -0.28222121]
Accuracy is 0.8454545454545455
 Average accuracy is  0.8592493744787323




pos 15
para [4e-07, 500]
fold 0 of model 29 finished [0.83413323 0.92594905 0.91596908 0.92654788 0.87884392 0.92415265
 0.8810394  0.92774545 0.93952185]
Accuracy is 0.29357798165137616


fold 1 of model 29 finished [0.8235545  0.92275551 0.91836432 0.92235633 0.87425318 0.9257495
 0.87744663 0.92375351 0.93812471]
Accuracy is 0.3577981651376147


fold 2 of model 29 finished [0.8265485  0.92375351 0.91636832 0.92155793 0.87784601 0.91996111
 0.87804548 0.92634831 0.93772551]
Accuracy is 0.39090909090909093


fold 3 of model 29 finished [0.83353456 0.92634836 0.91936238 0.92275559 0.87784608 0.92455196
 0.88183791 0.92654797 0.94051997]
Accuracy is 0.3119266055045872


fold 4 of model 29 finished [0.82874413 0.92594915 0.91716677 0.92195716 0.87804564 0.92714675
 0.88044073 0

fold 3 of model 37 finished [-0.3759866   1.60890617  0.52207241  0.3567132  -1.08979121  0.856545
 -0.65859853  0.41227824 -0.55030512]
Accuracy is 0.8073394495412844


fold 4 of model 37 finished [-0.38729714  1.74673527  0.18525758  0.22170746 -0.99069196  0.88643042
 -0.75314145  0.46251411 -0.34199916]
Accuracy is 0.8272727272727273
 Average accuracy is  0.8188990825688073




pos 56
para [3e-06, 1000]
fold 0 of model 38 finished [-0.55084352  0.28254094  0.17421138  0.25713282 -0.25601272  0.28811467
 -0.21623733  0.26481918  0.34906906]
Accuracy is 0.6697247706422018


fold 1 of model 38 finished [-0.58917863  0.27478806  0.23478892  0.23973464 -0.25901609  0.33216022
 -0.19883633  0.22407163  0.35556763]
Accuracy is 0.6788990825688074


fold 2 of model 38 finished [-0.57663283  0.28200141  0.21188234  0.21075234 -0.22352674  0.27216162
 -0.20332631  0.26670412  0.34274162]
Accuracy is 0.7454545454545455


fold 3 of model 38 finished [-0.5513057   0.28863831  0.22355799  0.20823

array([-0.367558  ,  1.46926394,  0.34006577,  0.32199452, -1.0234815 ,
        0.84304624, -0.68372141,  0.42888273, -0.35118253])

In [33]:
fit()#1st 

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 4 of model 2 finished [-0.39719284  1.36015213  0.30834254  0.27981433 -1.01277014  0.82084022
 -0.54934827  0.56831534 -0.31800174]
Accuracy is 0.8545454545454545
 Average accuracy is  0.8390325271059215




pos 80
para [8e-06, 500]
fold 0 of model 3 finished [-0.51987868  0.28610549  0.22472599  0.19321395 -0.27898861  0.32705659
 -0.24371016  0.27137349  0.2949857 ]
Accuracy is 0.7889908256880734


fold 1 of model 3 finished [-0.61804664  0.33771162  0.23668479  0.1995406  -0.29909657  0.35271399
 -0.20300868  0.2777064   0.31775438]
Accuracy is 0.6146788990825688


fold 2 of model 3 finished [-0.51449348  0.3045878   0.21942265  0.18127376 -0.29462954  0.33643627
 -0.23853383  0.25069781  0.30384412]
Accuracy is 0.8090909090909091


fold 3 of model 3 finished [-0.56159411  0.31640988  0.21706661  0.19386675 -0.29799326  0.40099236
 -0.26245143  0.26837843  0.30363861]
Accuracy is 0.6972477064220184


fold 4 of model 3 finished [-0.54594492  0.2782827   0.18247755  0.19139014 -

fold 2 of model 11 finished [-0.31484882  0.95632349  0.2744294   0.12044837 -0.74467115  0.50990933
 -0.59572332  0.36835759 -0.41943091]
Accuracy is 0.8545454545454545


fold 3 of model 11 finished [-0.44410886  1.1648844   0.08047714  0.10997471 -0.8751293   0.78967291
 -0.62724806  0.41882237 -0.28680928]
Accuracy is 0.8165137614678899


fold 4 of model 11 finished [-0.3493485   0.86649172  0.15662957  0.16380887 -0.70388946  0.5006271
 -0.4698512   0.32792769 -0.23237145]
Accuracy is 0.9545454545454546
 Average accuracy is  0.8627356130108425




pos 68
para [4.9999999999999996e-06, 2500]
fold 0 of model 12 finished [-0.35378472  0.46159944  0.28671318  0.12422448 -0.51704218  0.44738629
 -0.39192501  0.26884532 -0.01940829]
Accuracy is 0.8899082568807339


fold 1 of model 12 finished [-0.48726127  0.53572551  0.33785342  0.09035741 -0.53801569  0.4760613
 -0.33715758  0.26246086  0.02973152]
Accuracy is 0.7889908256880734


fold 2 of model 12 finished [-0.34246764  0.49804359  0.

fold 0 of model 20 finished [-0.33023758  0.54849757  0.29319021  0.12974882 -0.57989074  0.46552896
 -0.44212893  0.29104813 -0.10511945]
Accuracy is 0.8807339449541285


fold 1 of model 20 finished [-0.46745391  0.62866779  0.36263702  0.0871045  -0.61227151  0.49614481
 -0.38029425  0.27512014 -0.05453192]
Accuracy is 0.7981651376146789


fold 2 of model 20 finished [-0.31790514  0.58762685  0.31585501  0.06207585 -0.56623877  0.45194633
 -0.43051376  0.26813965 -0.1717738 ]
Accuracy is 0.8727272727272727


fold 3 of model 20 finished [-0.40595489  0.62664906  0.25705006  0.09512686 -0.62498909  0.61148757
 -0.44373414  0.29791897 -0.11317335]
Accuracy is 0.8165137614678899


fold 4 of model 20 finished [-0.35028608  0.53024607  0.24232496  0.13021499 -0.549435    0.44583164
 -0.34109951  0.25432992 -0.10263437]
Accuracy is 0.9636363636363636
 Average accuracy is  0.8663552960800667




pos 151
para [0.0004, 1000]
fold 0 of model 21 finished [-0.31215035  0.96274312  0.17834258  0.1

fold 4 of model 28 finished [-5.36239054  8.31094225  1.02830283  1.08882446 -8.24307769  4.98357767
 -4.91547649  3.19682458 -2.82929183]
Accuracy is 0.8181818181818182
 Average accuracy is  0.8244370308590492




pos 215
para [0.008, 500]
fold 0 of model 29 finished [ -5.83115019  13.24907071   3.93729933   2.19876693 -11.90369965
   8.76488139  -8.18291967   6.15505445  -3.62604089]
Accuracy is 0.8990825688073395


fold 1 of model 29 finished [-4.16517508 13.17764343  3.22810504  1.73364963 -8.99192297  6.46908937
 -4.48219491  3.79696018 -1.73160522]
Accuracy is 0.7431192660550459


fold 2 of model 29 finished [-3.87612096 13.98758224  3.81277991  1.54756159 -9.54270684  7.53034812
 -6.75314385  5.27250217 -5.49426128]
Accuracy is 0.8818181818181818


fold 3 of model 29 finished [-3.31914396 13.53545336  1.30359297  1.37933451 -8.39545687  8.97605131
 -5.16183395  5.0630162  -2.76643108]
Accuracy is 0.6697247706422018


fold 4 of model 29 finished [ -5.74914928  14.97168138   3.378

fold 2 of model 37 finished [-0.40356577  1.17047923  0.25921641  0.08591902 -0.94628996  0.59113255
 -0.71522811  0.39543963 -0.53538771]
Accuracy is 0.8636363636363636


fold 3 of model 37 finished [-0.4465674   1.28866297  0.03788825  0.11191058 -0.91770531  0.83703588
 -0.65734351  0.43820629 -0.29057264]
Accuracy is 0.8165137614678899


fold 4 of model 37 finished [-0.50759091  1.10665314  0.16509545  0.16046457 -0.99823927  0.61612393
 -0.61396965  0.41293896 -0.33954131]
Accuracy is 0.9454545454545454
 Average accuracy is  0.8737447873227691




pos 277
para [0.2, 1500]
fold 0 of model 38 finished [ -83.98957689  348.30253076  142.64491767  118.52871937 -202.15478102
  272.17265422 -126.95698593  202.31986297  -66.01317245]
Accuracy is 0.3853211009174312


fold 1 of model 38 finished [-104.34213878  307.09574386   71.56998211   39.11194848 -227.09045
  149.07438849 -112.02856205   91.29695659  -47.36634145]
Accuracy is 0.8165137614678899


fold 2 of model 38 finished [ -43.02706

array([-0.40361621,  1.43599484,  0.33778698,  0.20285568, -1.03196687,
        0.8596985 , -0.66696437,  0.56122939, -0.35381593])

In [3]:
    wfinal=findw(0.01,2000,X_train[0],y_train[0])
    a = predict(X_valid[0],wfinal)
    accu=accuracy(a,y_valid[0])
    co = conmat(a,y_valid[0])
    print(accu)
    print(co)

(109,)
(109,)
0.8073394495412844
[[42 21]
 [ 0 46]]


In [61]:
    wfinal=findw(0.01,2000,X_train[0],y_train[0])
    a = predict(X_valid[0],wfinal)
    accu=accuracy(a,y_valid[0])
    co = conmat(a,y_valid[0])
    print(accu)
    print(co)

(109,)
(109,)
0.8715596330275229
[[36  7]
 [ 7 59]]


In [41]:
wfinal=findw(0.001,1000,trainset[1])
a = predict(vertiset[1],wfinal)
accu=accuracy(a,vertiset[1])
print(wfinal)
print(accu)


[-0.68420535  1.68733281  0.5458797   0.20112867 -1.41705339  0.69981061
 -1.05910647  0.54854346 -0.54448549]
0.8256880733944955


In [42]:
#after shuffle
wfinal=findw(0.01,1000,data2)
a = predict(testset,wfinal)
accu=accuracy(a,testset)
print(wfinal)
print(accu)

[ -6.88006286  18.22786993   5.79209989   1.64802726 -15.11799986
  10.36724435  -9.91923968   7.55013216  -5.70546553]
0.8602941176470589


In [37]:
from sklearn.linear_model import LogisticRegression
X = trainset[1][:,0:data2.shape[1]-1]
y = trainset[1][:,data2.shape[1]-1]
clf = LogisticRegression().fit(X, y)
r = clf.predict(vertiset[1][:,0:data2.shape[1]-1])
accuracy(r,vertiset[1])

/home/mz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9541284403669725

In [19]:
x=testset[0:9,0:9]
print(x)
np.random.shuffle(x)
print(x)

[[ 5.  1.  1.  1.  2.  1.  3.  1.  1.]
 [ 5.  4.  4.  5.  7. 10.  3.  2.  1.]
 [ 3.  1.  1.  1.  2.  2.  3.  1.  1.]
 [ 6.  8.  8.  1.  3.  4.  3.  7.  1.]
 [ 4.  1.  1.  3.  2.  1.  3.  1.  1.]
 [ 8. 10. 10.  8.  7. 10.  9.  7.  1.]
 [ 1.  1.  1.  1.  2. 10.  3.  1.  1.]
 [ 2.  1.  2.  1.  2.  1.  3.  1.  1.]
 [ 2.  1.  1.  1.  2.  1.  1.  1.  5.]]
[[ 2.  1.  2.  1.  2.  1.  3.  1.  1.]
 [ 1.  1.  1.  1.  2. 10.  3.  1.  1.]
 [ 4.  1.  1.  3.  2.  1.  3.  1.  1.]
 [ 5.  4.  4.  5.  7. 10.  3.  2.  1.]
 [ 2.  1.  1.  1.  2.  1.  1.  1.  5.]
 [ 5.  1.  1.  1.  2.  1.  3.  1.  1.]
 [ 8. 10. 10.  8.  7. 10.  9.  7.  1.]
 [ 3.  1.  1.  1.  2.  2.  3.  1.  1.]
 [ 6.  8.  8.  1.  3.  4.  3.  7.  1.]]


In [5]:
fit()

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

array([-34.06410731,  78.51283051,  22.68621645,   4.63422952,
       -61.00099071,  59.01449698, -27.70283355,  32.15198299,
       -21.33453794])